In [1]:
import pandas as pd
pd.set_option('display.max_columns', 25)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.6f' % x) #为了直观的显示数字，不采用科学计数法
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei'] # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False # 用来正常显示负号
import numpy as np
np.set_printoptions(suppress = True)
import scipy
import os
import math
import time
import random
from joblib import Parallel, delayed
import warnings
from tqdm.notebook import tqdm

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
folder = '../compdata/4G5G_Data/提交文件样例/'
files = list(filter(lambda x: x[-3:] == 'csv', os.listdir(folder)))
files

['4g_PDCPDL_C48FDFBFC4072E0E.csv',
 '4g_PUSCH_C48FDFBFC4072E0E.csv',
 '4g_RRC_EA5EAA705108BDA0.csv',
 '5g_PDCPUL_F37F452354AC87C9.csv',
 '5g_RRC_EA5EAA705108BDA0.csv',
 '5g_PDCCH_F37F452354AC87C9.csv',
 '5g_PDCPDL_EA5EAA705108BDA0.csv',
 '4g_PDCCH_F37F452354AC87C9.csv',
 '4g_PDCPUL_EA5EAA705108BDA0.csv',
 '4g_PDSCH_AFE97F546A10368F.csv',
 '4g_PDSCH_EA5EAA705108BDA0.csv',
 '4g_PUSCH_F37F452354AC87C9.csv',
 '5g_PDCPUL_EA5EAA705108BDA0.csv',
 '5g_PDSCH_EA5EAA705108BDA0.csv',
 '5g_RRC_F37F452354AC87C9.csv',
 '5g_RRC_AFE97F546A10368F.csv',
 '5g_RRC_C48FDFBFC4072E0E.csv',
 '4g_PDCPUL_F37F452354AC87C9.csv',
 '5g_PDCPDL_AFE97F546A10368F.csv',
 '5g_PDCPDL_F37F452354AC87C9.csv',
 '5g_PDSCH_C48FDFBFC4072E0E.csv',
 '4g_PDCCH_C48FDFBFC4072E0E.csv',
 '4g_PDCPUL_AFE97F546A10368F.csv',
 '4g_RRC_AFE97F546A10368F.csv',
 '5g_PDCCH_C48FDFBFC4072E0E.csv',
 '5g_PDCCH_EA5EAA705108BDA0.csv',
 '5g_PDSCH_AFE97F546A10368F.csv',
 '5g_PUSCH_EA5EAA705108BDA0.csv',
 '4g_PDCCH_EA5EAA705108BDA0.csv',
 '5g_PDCCH_AFE97F

In [4]:
for file in files:
    userlabels = pd.read_csv(folder + file, encoding='GBK', usecols=['UserLabel']).drop_duplicates()
#     userlabels.to_pickle('preprocessed_data/Userlabel_lists_final/%s.pkl' % file[:-4])
    userlabels

,UserLabel
0,0000957df11d5c090b06fa4c7cb39b6b-2
1,0000957df11d5c090b06fa4c7cb39b6b-3
2,000658058cc54c9e2feac37361136352-3
3,000664643e6249a6f6fc4258fb2cd2fe-1
4,00117045b4c21fc5bd10991ecc5cf2af-104
...,...
22927,ffee9064f4297588d32dad8b5c000d0c-1
22928,ffee9064f4297588d32dad8b5c000d0c-2
22929,fff027a05b1f8a6150dcf7822826c305-201
22930,fff027a05b1f8a6150dcf7822826c305-3


,UserLabel
0,000658058cc54c9e2feac37361136352-3
1,000664643e6249a6f6fc4258fb2cd2fe-1
2,000a53c270b9ea0a1e9c900ac337c17a-2
3,000a53c270b9ea0a1e9c900ac337c17a-3
4,00117045b4c21fc5bd10991ecc5cf2af-104
...,...
22927,ffe6d3f1952b51d4449d3d2d050e209e-2
22928,fff027a05b1f8a6150dcf7822826c305-102
22929,fff027a05b1f8a6150dcf7822826c305-201
22930,fff027a05b1f8a6150dcf7822826c305-3


,UserLabel
0,0009423fca6ee800cc9b977c40864927-3
1,0012841fe2dc0967a74ab4f59a3974be-2
2,0012841fe2dc0967a74ab4f59a3974be-3
3,001ff91c00e59b98c503128065ddb378-2
4,004f9e20341211543ac11382059610c8-2
...,...
10791,ffdd7ddea3024b4cbcd6a7583efae05f-3
10792,ffe4d2a97d922e233d4452af4af0b0b1-2
10793,ffe857b2fc89030f97d8738aa1ff5cff-1
10794,ffe857b2fc89030f97d8738aa1ff5cff-2


,UserLabel
0,00bf8118a198a875528f956c6a426c6a-2
1,0258c15906d5dcda7c3604c023e17d7f-1
2,04b507aba5585bfa3effd058e8bf5989-3
3,0659da24be22b0d43b779f173a41a2f1-3
4,084f6db9476b0e9ea3358128b3a1a051-1
...,...
202,f81b7f89f2b75f8396de3bdf334c3a1c-1
203,f8c109d863829ccf7f565fc1fc86ba1c-2
204,fa31860d34e49a1de625329bd47e341a-3
205,fca6353eb0825143501d1d89bc13a914-3


,UserLabel
0,00853eb62caee38c654ec1d9b7258580-3
1,02abec48fb0f22bf8f34f3ae61c23211-1
2,02c9a98ef7d91e64997d8bc4f72b4625-2
3,03f4a59358246eeaeb3f1d65145eb3c8-1
4,0499de903c5514b847bdab1cbfbaed58-2
...,...
647,ffb6c60d96b1797cc321dfb71e775c16-1
648,ffbc00a6746d42cf47db9faace378202-1
649,ffbc00a6746d42cf47db9faace378202-3
650,ffc32bf3bcf7bfd0d685fcf7cda1f8a0-1


...

,UserLabel
0,0021d5a0c751023dfb91d5d933ef4cd5-1
1,00267b9ea4f3af222eaa6e17eb121f55-1
2,002e48d08c7763495263116429028d3c-3
3,0089fa8a29413b709ebefebd2a71e006-1
4,00b6a37a4a409aee963f9587ed3bbee9-2
...,...
1782,ff48a335b199400c8943f3f864693207-1
1783,ff4e2fe1d2d9136694c49ef8154f2718-3
1784,ff94070b2b363bb88caa39f1df1b3077-1
1785,ff94070b2b363bb88caa39f1df1b3077-2


,UserLabel
0,001586cce308d79ebeebb19778c96d88-3
1,003d860fec11f33a17ad366e50697b5e-1
2,003d860fec11f33a17ad366e50697b5e-2
3,003e1b5968c1d7aca5fa380bebf1dadc-103
4,003e1b5968c1d7aca5fa380bebf1dadc-3
...,...
5218,ffd8c0d4fc60dc114ea965755fa5f45f-1
5219,ffd8c0d4fc60dc114ea965755fa5f45f-3
5220,ffe41a7065e57932494e595cebdb1383-2
5221,ffe41a7065e57932494e595cebdb1383-3


,UserLabel
0,000434ae1d9f8682cbef64acc4675ac3-1
1,000434ae1d9f8682cbef64acc4675ac3-2
2,0004e3ed31b2a98fe365a62b59d2cae3-2
3,0009ca3a59a2ca50c26970d319a3d9a0-103
4,0009ca3a59a2ca50c26970d319a3d9a0-2
...,...
24369,ffe31bcd25e9a1ef3cbe580b014abb9f-3
24370,ffe4e3ccba1f48ad654230e5bd9b0c60-101
24371,ffe9de56942ec25c21caa30fbe05ca5b-1
24372,ffedd6894f34979b8d91974d2466417e-2


,UserLabel
0,0000957df11d5c090b06fa4c7cb39b6b-2
1,0000957df11d5c090b06fa4c7cb39b6b-3
2,000658058cc54c9e2feac37361136352-3
3,000664643e6249a6f6fc4258fb2cd2fe-1
4,000a53c270b9ea0a1e9c900ac337c17a-2
...,...
22927,ffe6d3f1952b51d4449d3d2d050e209e-1
22928,ffe6d3f1952b51d4449d3d2d050e209e-2
22929,ffee9064f4297588d32dad8b5c000d0c-1
22930,fff027a05b1f8a6150dcf7822826c305-3


,UserLabel
0,00bf8118a198a875528f956c6a426c6a-2
1,0258c15906d5dcda7c3604c023e17d7f-1
2,044eed89e34909d5eeefa14113029e1a-1
3,0659da24be22b0d43b779f173a41a2f1-1
4,084f6db9476b0e9ea3358128b3a1a051-3
...,...
202,f81b7f89f2b75f8396de3bdf334c3a1c-1
203,f8c109d863829ccf7f565fc1fc86ba1c-2
204,fa31860d34e49a1de625329bd47e341a-1
205,fa31860d34e49a1de625329bd47e341a-3
